# RAG Pipeline

This notebook is our pipeline to transform the various websites around CS to text and put them in the vector store.

In [1]:
import chromadb
from chromadb.utils.embedding_functions import Bm25EmbeddingFunction, DefaultEmbeddingFunction
from chromadb import Documents, EmbeddingFunction, Embeddings
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os 

load_dotenv()

True

In [2]:
URL = "https://api.infomaniak.com/1/ai/models"
headers = {
  'Authorization': f'Bearer {os.getenv("API_TOKEN")}',
  'Content-Type': 'application/json',
}
req = requests.request("GET", url = URL , headers = headers)
res = req.json()

info_df = pd.DataFrame(res["data"])
info_df[info_df["type"] == "embedding"]

,id,name,type,documentation_link,description,info_status,logo_url,last_updated_at,max_token_input,version,meta
6,12,bge_multilingual_gemma2,embedding,https://developer.infomaniak.com/docs/api/post...,Bge Multilingual Gemma2,coming_soon,https://storage4.infomaniak.com/ai-tools/publi...,2024-12-04,8000.0,1.0,{'is_beta': False}
7,13,mini_lm_l12_v2,embedding,https://developer.infomaniak.com/docs/api/post...,All MiniLM L12 v2,coming_soon,https://storage4.infomaniak.com/ai-tools/publi...,2024-12-04,128.0,2.0,{'is_beta': False}


In [3]:
from dotenv import load_dotenv
import chromadb
import os

load_dotenv() 

chroma_client = chromadb.Client()

In [4]:
collection = chroma_client.get_or_create_collection(name="main")
collection

Collection(name=main)

In [5]:
chroma_client.list_collections()

[Collection(name=main)]

In [6]:
# just a quick demo on how this works
# by default this uses a MiniLM L6-v2
collection.add(
    ids=["id1", "id2", "id3", "id4"],
    documents=[
        "document sur un cfc / apprentissage",
        "document sur l'epfl",
        "document sur 42 Lausanne",
        "document sur université de fribougr",
    ]
)

print("Number of items in the collection:", collection.count())

Number of items in the collection: 4


In [7]:
out = collection.query(
    query_texts=["J'aimerais faire un apprentissage"],
    n_results=1,
)

out

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['document sur un cfc / apprentissage']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None]],
 'distances': [[0.9254448413848877]]}

In [8]:
out = collection.query(
    query_texts=["J'aimerais aller au collège pour ensuite faire l'université"],
    n_results=2,
)

# key documents returns the text of the relevant docs
out["documents"]

[['document sur université de fribougr', "document sur l'epfl"]]

In [9]:
chroma_client.delete_collection(name="main")

## 1. Data Processing: Conver HTML to Text

using beautifulSoup and simple get request, we collect the data from websites.

In [10]:
# needed to pretend I am a normal user ;)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
}

def url_to_string(url: str) -> str:
    if not url.startswith("https://"):
        url = "https://" + url
    
    res = requests.get(url, headers=HEADERS)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "html.parser")
    site_text = " ".join([text for text in soup.stripped_strings])
    return site_text


demo_url = "www.orientation.ch/dyn/show/1900?id=152"
site_text = url_to_string(demo_url)

print("Number of chars:", len(site_text))
print("Number of words:", len(site_text.split()))

Number of chars: 30161
Number of words: 3883


In [11]:
df = pd.read_csv("../links.csv")

url_list = df["link"].to_list()
url_text_list = [url_to_string(url) for url in url_list]

In [12]:
df["url_text"] = url_text_list
df.head()

,title,link,url_text
0,informaticen cfc,www.orientation.ch/dyn/show/1900?id=152,Informaticien CFC / Informaticienne CFC - orie...
1,informaticen es,www.orientation.ch/dyn/show/1900?id=885,Informaticien ES / Informaticienne ES - orient...
2,informatique université de fribourg,www.unifr.ch/inf/fr/informatique,Informatique | Département d'informatique | U...
3,informatique de gestion université de fribourg,www.unifr.ch/inf/fr/informatique-de-gestion,Informatique de gestion | Département d'inform...
4,informatique université de genève,www.unige.ch/dinfo/formations/bachelor,Bachelor en sciences informatiques Présentatio...


## 2. RAG Benchmark

First two models are on infomaniak.
The rest we will need to train ourselves by getting the model with a GPU.

|evaluate? | Model name | Link |
|:--------:|------------|------|
| |MiniLM L12 v2 | https://developer.infomaniak.com/docs/api/post/1/ai/%7Bproduct_id%7D/openai/v1/embeddings |
| X |BGE Multilingual Gemma 2 | https://developer.infomaniak.com/docs/api/post/1/ai/%7Bproduct_id%7D/openai/v1/embeddings |
| X | paraphrase-multilingual-MiniLM-L12-v2 | https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 |
| X | paraphrase-multilingual-mpnet-base-v2 | https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2 |
| X | distiluse-base-multilingual-cased-v2 | https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2 |
|  | Alibaba-NLP/gte-Qwen2-1.5B-instruct | https://huggingface.co/Alibaba-NLP/gte-Qwen2-1.5B-instruct |
| X | BM25 | chromadb through `fastembed` | 

In [13]:
DB_PATH = "../data/"

client = chromadb.PersistentClient(path=DB_PATH)
# ensure the DB is available
print("hearbeat:", client.heartbeat())

ids: list[str] = df["title"].to_list() #df.index.astype("str").to_list()
documents: list[str] = df["url_text"].to_list()

hearbeat: 1760201144161595907


## Infomaniak Model: MultiLingual Gemma 2 and MiniLM-L12-v2

The latter is english only, we should not evaluate it.

In [14]:
PRODUCT_ID = os.getenv("PRODUCT_ID")
API_TOKEN = os.getenv("API_TOKEN")

URL = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}/openai/v1/embeddings"

headers = {
  'Authorization': f"Bearer {API_TOKEN}",
  'Content-Type': 'application/json',
}

payload = {
    "input": documents,
    "model": "bge_multilingual_gemma2",
    "mode": "index",
}

req = requests.post(url=URL , json=payload, headers=headers)
res = req.json()
print(req.status_code)

assert len(documents) == len(res["data"])

200


In [15]:
PRODUCT_ID = os.getenv("PRODUCT_ID")
API_TOKEN = os.getenv("API_TOKEN")


class MultinligualGemma2(EmbeddingFunction):
    def __init__(self) -> None:
        self.model = "bge_multilingual_gemma2"
        self.url = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}/openai/v1/embeddings"
        self.headers = {
          'Authorization': f"Bearer {API_TOKEN}",
          'Content-Type': 'application/json',
        }
        
    def __call__(self, input_data: Documents) -> Embeddings:
        payload = {
            "input": input_data,
            "model": self.model,
        }

        req = requests.post(url=self.url, json=payload, headers=self.headers)
        res = req.json()
        data = res["data"]
        embeddings = [np.array(x["embedding"]) for x in data]
        
        return embeddings


class MiniLML12V2(EmbeddingFunction):
    def __init__(self) -> None:
        self.model = "mini_lm_l12_v2"
        self.url = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}/openai/v1/embeddings"
        self.headers = {
          'Authorization': f"Bearer {API_TOKEN}",
          'Content-Type': 'application/json',
        }
        
    def __call__(self, input_data: Documents) -> Embeddings:
        payload = {
            "input": input_data,
            "model": self.model,
        }

        req = requests.post(url=self.url, json=payload, headers=self.headers)
        res = req.json()
        data = res["data"]
        embeddings = [np.array(x["embedding"]) for x in data]
        
        return embeddings

In [16]:
multilingual_gemma2_collection = client.get_or_create_collection(
    name="multilingual-gemma2",
    embedding_function=MultinligualGemma2()
)

print(multilingual_gemma2_collection)
print(multilingual_gemma2_collection._embedding_function)

Collection(name=multilingual-gemma2)


In [17]:
multilingual_gemma2_collection.add(
    ids=ids,
    documents=documents,
)

In [18]:
multilingual_gemma2_collection.count()

8

In [19]:
results = multilingual_gemma2_collection.query(
    query_texts=["J'aimerais faire un apprentissage"],
    n_results=2,
)   

In [20]:
results["ids"]

[['42 lausanne', 'informaticen cfc']]

## BM25 

In [23]:
bm_25_collection = client.get_or_create_collection(
    name="bm25",
    embedding_function=Bm25EmbeddingFunction()
)

print(bm_25_collection)
print(bm_25_collection._embedding_function)

/home/matt/Projects/py/hackhaton_lab/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Collection(name=bm25)


In [24]:
ids: list[str] = df["title"].to_list() #df.index.astype("str").to_list()
documents: list[str] = df["url_text"].to_list()

# NOT WOKRING: WHY?
bm_25_collection.add(
    ids=ids,
    documents=documents,
)

TypeError: argument 'embeddings': 'dict' object cannot be converted to 'Sequence'